In [1]:
#RUN ONLY THE FIRST TIME
%pip install pandas openpyxl

     -------------------------------------- 242.1/242.1 kB 1.9 MB/s eta 0:00:00

[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
from typing import List
from pathlib import Path, PureWindowsPath
from template import build_html
from emails import send_email

In [11]:
data_folder = Path.home()/"Desktop"
data_folder = data_folder/"autopony"
data_folder = data_folder/"test2022"
pdf_folder = Path.home()/"Desktop"
pdf_folder = pdf_folder/"LEGACY FILES"
EXEL_FILE_DATA = 'TEST.xlsx'
EXEL_FILE_EMAILS = 'TEST.xlsx'
sheet_data = "DATA"
sheet_emails = "EMAILS"
custome_subject = "IBM - "

In [12]:
#Read excel files
customers_file = data_folder/EXEL_FILE_EMAILS
#customers_file = PureWindowsPath(customers_file)

data_file = f'{data_folder}\{EXEL_FILE_DATA}'

df_customers = pd.read_excel(customers_file, sheet_name=sheet_emails)
df_data = pd.read_excel(customers_file, sheet_name=sheet_data)


In [13]:
#Query function
def query(col_name:str, search:str)-> tuple:
    df_tmp = df_data[df_data[col_name] == search]
    #exclude columns
    df_tmp = df_tmp[["Enterprise #", "Customer Name","Customer #","Invoice #", "Inv. Date", "Inv. Amount","Inv. Balance"]]
    pdfs_names = df_tmp["Invoice #"].tolist()
    return df_tmp.to_html(), pdfs_names

In [14]:
#building attachments paths
def prepare_path(file:str):
    tmp_f = pdf_folder/f'{str(file)}.pdf'
    return str(tmp_f.absolute())

def build_attachments(pdfs:List[str])->List[str]:
    tmp_paths = list( map(prepare_path , pdfs )) 
    return tmp_paths

In [15]:
# Iterate Customers & send emails
for index, row in df_customers.iterrows():
    table, pdfs = query(col_name='Customer #',search=row['Customer #'])
    customer_email = row['email']
    customer_name = row['Customer Name']
    custome_subject = F'{custome_subject} {customer_name}'
    body = build_html(table, row['Customer Name'])
    path_atachments = build_attachments(pdfs)
    send_email(customer_email,body, path_atachments, custome_subject)